In [14]:
import os
import pandas as pd
import sys
from interpret import show
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.model_selection import train_test_split
from interpret.perf import RegressionPerf
from interpret.data import ClassHistogram, Marginal

In [2]:
in_file = r"E:\work\results\grid_search\metrics.csv"
in_file = os.path.abspath(in_file)
in_dir = os.path.dirname(in_file)
random_state = 42

df = pd.read_csv(in_file)

df

,image,class,threshold,metric,value,data_dataset,model_activation,model_depth,model_in_channels,model_out_classes,model_residual_units,model_width,model_width_ratio,job_hash,epoch,loss,predict_time
0,bremen_000075_000019,road,0.5,iou,0.546288,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,496.421014
1,bremen_000075_000019,sidewalk,0.5,iou,0.824461,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,496.421014
2,bremen_000075_000019,building,0.5,iou,0.593378,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,496.421014
3,bremen_000075_000019,fence,0.5,iou,0.533155,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,496.421014
4,bremen_000075_000019,pole,0.5,iou,0.462896,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,496.421014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628283,lindau_000018_000019,vegetation,0.5,recall,0.966705,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,470.325362
2628284,lindau_000018_000019,building,0.5,precision,0.681317,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,470.325362
2628285,lindau_000018_000019,pole,0.5,precision,0.010003,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,470.325362
2628286,lindau_000018_000019,traffic sign,0.5,precision,0.549224,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,470.325362


In [3]:
grouped = df.groupby(
    by=["job_hash", "class", "metric"]).mean().reset_index()

mean_grouped = df.groupby(by=["job_hash", "metric", "model_activation", "model_depth", "model_in_channels", "model_out_classes", "model_residual_units", "model_width", "model_width_ratio"]).mean().reset_index()

iou_df = mean_grouped[mean_grouped.metric == "iou"].sort_values("value")

iou_df

,job_hash,metric,model_activation,model_depth,model_in_channels,model_out_classes,model_residual_units,model_width,model_width_ratio,threshold,value,epoch,loss,predict_time
174,50212d16b7ce97c296683eb47f24a9ec,iou,tanh,4,3,19,2,30,1.2,0.5,0.290458,8,0.560493,719.626741
225,6c1651c812ecbef9552834b9f218fdb6,iou,tanh,4,3,19,2,50,1.4,0.5,0.297651,15,0.529416,663.168734
264,7ffd8659e11504fe9fc7fc443208bdf7,iou,piecewise_linear,4,3,19,2,30,1.4,0.5,0.355125,59,0.453266,704.976407
78,28c3786b195e86762531ade286c3f9f8,iou,cos,4,3,19,2,30,1.2,0.5,0.359161,71,0.456369,594.813869
87,2a85f5ef5ca0aaca52446cc87d8e3e9c,iou,piecewise_linear,4,3,19,2,30,1.2,0.5,0.360737,67,0.455198,614.620386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0c9b3e43ee7d293529e9c5da1cb72ce4,iou,leaky_relu,6,3,19,4,50,1.4,0.5,0.522328,163,0.289866,536.497332
51,1ec1b3cdde28b001830e9d07ac190d3d,iou,swish,8,3,19,4,50,1.4,0.5,0.524180,86,0.285425,419.054161
228,6e8912b429699396a12723b2bcaeff78,iou,relu,8,3,19,4,50,1.2,0.5,0.526533,91,0.285530,411.274735
333,bfd33d38ce9c4d84b912f1ee0b7961aa,iou,leaky_relu,8,3,19,4,50,1.4,0.5,0.529178,79,0.279563,423.991765


In [4]:
iou_df.columns

Index(['job_hash', 'metric', 'model_activation', 'model_depth',
       'model_in_channels', 'model_out_classes', 'model_residual_units',
       'model_width', 'model_width_ratio', 'threshold', 'value', 'epoch',
       'loss', 'predict_time'],
      dtype='object')

In [15]:
#train_cols = ['model_depth', 'model_residual_units', 'model_width', 'model_width_ratio', 'model_activation', "class", "image"]
train_cols = ['model_depth', 'model_residual_units', 'model_width', 'model_width_ratio', 'model_activation']
label = "value"

X = iou_df[train_cols]
y = iou_df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random_state)

train_data_hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data Histogram')
train_data_marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data Marginal Plot')

In [6]:
ebm = ExplainableBoostingRegressor(
    feature_names=train_cols,
    feature_types=['categorical'] * len(train_cols),
    max_bins=256,
    max_interaction_bins=32,
    binning="quantile",
    # Stages
    mains="all",
    interactions=4,
    # Ensemble
    outer_bags=8,
    inner_bags=0,
    # Boosting
    learning_rate=0.01,
    validation_size=0.10,
    early_stopping_rounds=50,
    early_stopping_tolerance=1e-4,
    max_rounds=sys.maxsize,
    # Trees
    min_samples_leaf=2,
    max_leaves=3,
    # Overall
    n_jobs=os.cpu_count(),
    random_state=42,
)
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['model_depth',
                                            'model_residual_units',
                                            'model_width', 'model_width_ratio',
                                            'model_activation',
                                            'model_depth x '
                                            'model_residual_units',
                                            'model_residual_units x '
                                            'model_activation',
                                            'model_depth x model_width',
                                            'model_depth x model_activation'],
                             feature_types=['categorical', 'categorical',
                                            'categorical', 'categorical',
                                            'categorical', 'interaction',
                                            'interaction', 'interaction',
                  

In [16]:
ebm_global = ebm.explain_global(name="IoU Predictor")
ebm_local = ebm.explain_local(X_test, y_test, name="Local IoU Predictor")
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')
show([train_data_hist, train_data_marginal, ebm_perf, ebm_local, ebm_global], share_tables=True)

<!-- http://127.0.0.1:7001/1986889035528/ -->
 Open in new window

In [8]:
ebm_score = ebm.score(X_test, y_test)
ebm_score

0.9206462784217954